<a href="https://colab.research.google.com/github/jfdelgad/convolutionalnetworks_mnist/blob/master/deepConvNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import tools
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt


# Load data and normalize to make images values have amaximun of 1
(trainData, trainLabels),(testData, testLabels) = mnist.load_data();
trainData = trainData.astype('float32')/255
testData = testData.astype('float32')/255

trainData = np.expand_dims(trainData, 3);
testData = np.expand_dims(testData, 3);

trainLabels = keras.utils.to_categorical(trainLabels,10);
testLabels = keras.utils.to_categorical(testLabels,10);

print('Train data size: ', trainData.shape)
print('Test data size: ', testData.shape)

In [ ]:
# if you are rerunning this, clear the old model
keras.backend.clear_session()

# Set the network 3 conv2d layers with maxpooling over the size of the filter of the previous layer
# also batchnormalization is used before every layer, which imporves speed
nputs = keras.layers.Input(shape=(trainData.shape[1:]))
x = keras.layers.BatchNormalization()(inputs)
x = keras.layers.Conv2D(100,3, activation='relu', padding='valid',input_shape=inputs.shape[1:])(x)
x = keras.layers.MaxPooling2D((3,3),(1,1))(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Conv2D(50,3, activation='relu')(x)
x = keras.layers.MaxPooling2D((3,3),(1,1))(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Conv2D(25,3, activation='relu')(x)
x = keras.layers.MaxPooling2D((3,3),(1,1))(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(100,activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(50,activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
outputs = keras.layers.Dense(10,activation='softmax')(x)

model = keras.Model(inputs=inputs,outputs=outputs)
model.summary()



In [ ]:
# Compile, use Adadelta to reduce the effect of decreasing earning rate
model.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adadelta(learning_rate=1),
    metrics=["accuracy"],
)

# train the model and plot the loss history, consider to increase the epochs if the loss history sugest it
history = model.fit(trainData,trainLabels,batch_size=1000, epochs=30, validation_split=0.2)
plt.plot(history.history['loss'])



In [ ]:
#Test the model
model.evaluate(testData,testLabels)